# Генеративные задачи в NLP II

# Методы RAG: поэтапное тестирование конфигураций
## Тестовый пайплайн с расширенной оценкой

Этот проект создан для понимания того, как различные настройки влияют на системы Retrieval-Augmented Generation (RAG). Мы построим и протестируем пайплайн шаг за шагом с использованием **Nebius AI API**.

**Что предстоит выяснить:**
*   Как нарезка текста (`chunk_size`, `chunk_overlap`) влияет на то, какие данные извлекает система RAG.
*   Как количество извлекаемых документов (`top_k`) влияет на контекст, передаваемый LLM.
*   В чем разница между тремя распространёнными стратегиями RAG (Simple, Query Rewrite, Rerank).
*   Как использовать LLM (например, Nebius AI) для автоматической оценки качества сгенерированных ответов по нескольким метрикам: **Достоверность** (Faithfulness), **Релевантность** (Relevancy) и **Семантическое сходство** (Semantic Similarity) с эталонным ответом.
*   Как объединить эти метрики в средний балл для удобства сравнения.

Мы сосредоточимся на понимании *зачем* выполняется каждый шаг и будем чётко наблюдать результаты, с подробными объяснениями и комментированным кодом.


### Оглавление
1.  **Настройка: Установка библиотек**: Получаем необходимые инструменты.
2.  **Настройка: Импорт библиотек**: Подключаем инструменты в рабочее пространство.
3.  **Конфигурация: Подготовка эксперимента**: Определяем параметры API, модели, запросы для оценки и параметры тестирования.
4.  **Входные данные: Источник знаний и наш вопрос**: Определяем документы, из которых будет учиться система RAG, и формулируем вопрос.
5.  **Ключевой компонент: Функция нарезки текста**: Создаем функцию для разбиения документов на небольшие части.
6.  **Ключевой компонент: Подключение к Nebius AI**: Устанавливаем соединение для использования моделей Nebius.
7.  **Ключевой компонент: Функция косинусного сходства**: Создаем функцию для измерения семантического сходства между текстами.
8.  **Эксперимент: Перебор конфигураций**: Основной цикл, в котором тестируются разные настройки.
    *   8.1 Обработка конфигурации нарезки (Chunk, Embed, Index)
    *   8.2 Тестирование стратегий RAG для значения `top_k`
    *   8.3 Запуск и оценка отдельной стратегии RAG (включая сходство)
9.  **Анализ: Анализ результатов**: Используем Pandas для организации и отображения результатов.
10. **Выводы: Что мы узнали?**: Рефлексия по итогам и возможные следующие шаги.


### 1. Настройка: Установка библиотек

Сначала нужно установить Python-библиотеки, необходимые для работы в этом ноутбуке.
- `openai`: Для взаимодействия с Nebius API (он использует совместимый с OpenAI интерфейс).
- `pandas`: Для создания и управления таблицами данных (DataFrame).
- `numpy`: Для числовых операций, особенно с векторами (эмбеддингами).
- `faiss-cpu`: Для эффективного поиска похожих векторов (этап извлечения информации).
- `ipywidgets`, `tqdm`: Для отображения индикаторов выполнения в Jupyter.
- `scikit-learn`: Для расчёта косинусного сходства.


In [1]:
# Установка библиотек (запустите эту ячейку только один раз при необходимости)
!pip install openai pandas numpy faiss-cpu ipywidgets tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.5 MB/s eta 0:00:00


**Обратите внимание!** После завершения установки может потребоваться **перезапустить ядро** (или среду выполнения), чтобы Jupyter/Colab распознал новые библиотеки.


### 2. Настройка: Импорт библиотек

После установки библиотек импортируем их в наше Python-окружение, чтобы сделать их функции доступными для использования.


In [3]:
import os                     # Для доступа к переменным окружения (например, API-ключи)
import time                   # Для замера времени выполнения операций
import re                     # Для работы с регулярными выражениями (очистка текста)
import warnings               # Для управления предупреждающими сообщениями
import itertools              # Для удобного создания комбинаций параметров
import getpass                # Для безопасного ввода API-ключей, если они не заданы

import numpy as np            # Числовая библиотека для работы с векторами
import pandas as pd           # Библиотека для обработки и анализа таблиц (DataFrame)
import faiss                  # Библиотека для быстрого поиска схожих векторов
from openai import OpenAI     # Клиентская библиотека для взаимодействия с Nebius API
from tqdm.notebook import tqdm # Для отображения прогресс-баров
from sklearn.metrics.pairwise import cosine_similarity # Для расчёта метрики схожести

# Настройка параметров отображения для Pandas DataFrame для удобства чтения
pd.set_option('display.max_colwidth', 150) # Показывать больше текста в ячейках таблицы
pd.set_option('display.max_rows', 100)     # Показывать больше строк в таблицах
warnings.filterwarnings('ignore', category=FutureWarning) # Отключить определённые некритичные предупреждения

print("Библиотеки успешно импортированы!")

Библиотеки успешно импортированы!


### 3. Конфигурация: Настройка эксперимента

Здесь мы определяем все настройки и параметры для нашего эксперимента напрямую как переменные Python. Это удобно — вся конфигурация собрана в одном месте, её легко просматривать и изменять.

**Основные блоки конфигурации:**
*   **Данные для Nebius API:** Учётные данные и идентификаторы моделей для подключения к Nebius AI.
*   **Параметры LLM:** Настройки, управляющие поведением языковой модели при генерации ответа (например, `temperature` — уровень креативности).
*   **Промпты для оценки:** Конкретные инструкции (промпты), которые LLM получает для оценки Достоверности и Релевантности.
*   **Параметры для настройки:** Различные значения для размера чанка, перекрытия и количества извлекаемых документов (`top_k`), которые мы будем систематически тестировать.
*   **Настройки ранжирования:** Конфигурация для имитации стратегии повторного ранжирования.


In [4]:
# --- Конфигурация API NebiusAI ---
# Рекомендуется хранить API-ключи в переменных окружения, а не прописывать их напрямую в коде.
# Укажите свой настоящий ключ здесь или задайте его как переменную окружения
NEBIUS_API_KEY = os.getenv('NEBIUS_API_KEY', None)  # Загружаем API-ключ из переменной окружения
if NEBIUS_API_KEY is None:
    print("Внимание: NEBIUS_API_KEY не задан. Пожалуйста, установите его в переменных окружения или укажите напрямую в коде.")
NEBIUS_BASE_URL = "https://api.studio.nebius.com/v1/"
NEBIUS_EMBEDDING_MODEL = "BAAI/bge-multilingual-gemma2"  # Модель для преобразования текста в векторные эмбеддинги
NEBIUS_GENERATION_MODEL = "deepseek-ai/DeepSeek-V3"      # LLM для генерации итоговых ответов
NEBIUS_EVALUATION_MODEL = "deepseek-ai/DeepSeek-V3"      # LLM для оценки сгенерированных ответов

# --- Параметры генерации текста (для генерации ответа RAG) ---
GENERATION_TEMPERATURE = 0.1  # Меньшие значения (например, 0.1-0.3) делают ответ более точным и менее случайным, хорошо для фактических ответов.
GENERATION_MAX_TOKENS = 400   # Максимальное количество токенов (примерно слов/частей слов) в сгенерированном ответе.
GENERATION_TOP_P = 0.9        # Параметр nucleus sampling (обычно достаточно значения по умолчанию).

# --- Промпты для оценки (инструкции для оценщика LLM) ---
# Достоверность: Насколько ответ соответствует предоставленному контексту?
FAITHFULNESS_PROMPT = """
System: You are an objective evaluator. Evaluate the faithfulness of the AI Response compared to the True Answer, considering only the information present in the True Answer as the ground truth.
Faithfulness measures how accurately the AI response reflects the information in the True Answer, without adding unsupported facts or contradicting it.
Score STRICTLY using a float between 0.0 and 1.0, based on this scale:
- 0.0: Completely unfaithful, contradicts or fabricates information.
- 0.1-0.4: Low faithfulness with significant inaccuracies or unsupported claims.
- 0.5-0.6: Partially faithful but with noticeable inaccuracies or omissions.
- 0.7-0.8: Mostly faithful with only minor inaccuracies or phrasing differences.
- 0.9: Very faithful, slight wording differences but semantically aligned.
- 1.0: Completely faithful, accurately reflects the True Answer.
Respond ONLY with the numerical score.

User:
Query: {question}
AI Response: {response}
True Answer: {true_answer}
Score:"""

# Релевантность: Насколько ответ напрямую отвечает на вопрос пользователя?
RELEVANCY_PROMPT = """
System: You are an objective evaluator. Evaluate the relevance of the AI Response to the specific User Query.
Relevancy measures how well the response directly answers the user's question, avoiding unnecessary or off-topic information.
Score STRICTLY using a float between 0.0 and 1.0, based on this scale:
- 0.0: Not relevant at all.
- 0.1-0.4: Low relevance, addresses a different topic or misses the core question.
- 0.5-0.6: Partially relevant, answers only a part of the query or is tangentially related.
- 0.7-0.8: Mostly relevant, addresses the main aspects of the query but might include minor irrelevant details.
- 0.9: Highly relevant, directly answers the query with minimal extra information.
- 1.0: Completely relevant, directly and fully answers the exact question asked.
Respond ONLY with the numerical score.

User:
Query: {question}
AI Response: {response}
Score:"""

# --- Настраиваемые параметры (экспериментальные переменные) ---
CHUNK_SIZES_TO_TEST = [150, 250]    # Список размеров чанков (в словах) для тестирования.
CHUNK_OVERLAPS_TO_TEST = [30, 50]   # Список перекрытий чанков (в словах) для тестирования.
RETRIEVAL_TOP_K_TO_TEST = [3, 5]    # Список значений 'k' (количество чанков для поиска), которые будем тестировать.

# --- Конфигурация повторного ранжирования (используется только для стратегии Rerank) ---
RERANK_RETRIEVAL_MULTIPLIER = 3 # Для симулированного ранжирования: сначала извлекаем K * multiplier чанков.

# --- Проверка API-ключа ---
print("--- Проверка конфигурации --- ")
print(f"Пробуем загрузить Nebius API Key из переменной окружения 'NEBIUS_API_KEY'...")
if not NEBIUS_API_KEY:
    print("Nebius API Key не найден в переменных окружения.")
    # Запросить ключ у пользователя, если не найден.
    NEBIUS_API_KEY = getpass.getpass("Пожалуйста, введите свой Nebius API Key: ")
else:
    print("Nebius API Key успешно загружен из переменной окружения.")

# Вывод краткой информации о настройках для проверки
print(f"Модели: Embed='{NEBIUS_EMBEDDING_MODEL}', Gen='{NEBIUS_GENERATION_MODEL}', Eval='{NEBIUS_EVALUATION_MODEL}'")
print(f"Тестируемые размеры чанков: {CHUNK_SIZES_TO_TEST}")
print(f"Тестируемые значения перекрытия: {CHUNK_OVERLAPS_TO_TEST}")
print(f"Тестируемые значения Top-K: {RETRIEVAL_TOP_K_TO_TEST}")
print(f"Температура генерации: {GENERATION_TEMPERATURE}, Макс. токенов: {GENERATION_MAX_TOKENS}")
print("Конфигурация готова.")
print("-" * 25)


Внимание: NEBIUS_API_KEY не задан. Пожалуйста, установите его в переменных окружения или укажите напрямую в коде.
--- Проверка конфигурации --- 
Пробуем загрузить Nebius API Key из переменной окружения 'NEBIUS_API_KEY'...
Nebius API Key не найден в переменных окружения.
Пожалуйста, введите свой Nebius API Key: ··········
Модели: Embed='BAAI/bge-multilingual-gemma2', Gen='deepseek-ai/DeepSeek-V3', Eval='deepseek-ai/DeepSeek-V3'
Тестируемые размеры чанков: [150, 250]
Тестируемые значения перекрытия: [30, 50]
Тестируемые значения Top-K: [3, 5]
Температура генерации: 0.1, Макс. токенов: 400
Конфигурация готова.
-------------------------


### 4. Входные данные: Источник знаний и наш вопрос

Любая система RAG нуждается в базе знаний, из которой будет извлекаться информация. Здесь мы определяем:
*   `corpus_texts`: Список строк, где каждая строка — это документ с информацией (в данном случае, об источниках возобновляемой энергии).
*   `test_query`: Конкретный вопрос, на который мы хотим получить ответ от системы RAG, используя `corpus_texts`.
*   `true_answer_for_query`: Тщательно сформулированный «эталонный» ответ, основанный *только* на информации из `corpus_texts`. Это важно для точной оценки достоверности (Faithfulness) и семантического сходства (Semantic Similarity).

In [6]:
# Наша база знаний: список текстовых документов об альтернативной энергетике
corpus_texts = [
    "Solar power uses PV panels or CSP systems. PV converts sunlight directly to electricity. CSP uses mirrors to heat fluid driving a turbine. It's clean but varies with weather/time. Storage (batteries) is key for consistency.", # Документ 0
    "Wind energy uses turbines in wind farms. It's sustainable with low operating costs. Wind speed varies, siting can be challenging (visual/noise). Offshore wind is stronger and more consistent.", # Документ 1
    "Hydropower uses moving water, often via dams spinning turbines. Reliable, large-scale power with flood control/water storage benefits. Big dams harm ecosystems and displace communities. Run-of-river is smaller, less disruptive.", # Документ 2
    "Geothermal energy uses Earth's heat via steam/hot water for turbines. Consistent 24/7 power, small footprint. High initial drilling costs, sites are geographically limited.", # Документ 3
    "Biomass energy from organic matter (wood, crops, waste). Burned directly or converted to biofuels. Uses waste, provides dispatchable power. Requires sustainable sourcing. Combustion releases emissions (carbon-neutral if balanced by regrowth)." # Документ 4
]

# Вопрос, который мы зададим системе RAG
test_query = "Compare the consistency and environmental impact of solar power versus hydropower."

# !!! ВАЖНО: 'True Answer' ДОЛЖЕН быть выведен ТОЛЬКО из corpus_texts выше !!!
# Это наш эталон для оценки.
true_answer_for_query = "Solar power's consistency varies with weather and time of day, requiring storage like batteries. Hydropower is generally reliable, but large dams have significant environmental impacts on ecosystems and communities, unlike solar power's primary impact being land use for panels."

print(f"Загружено {len(corpus_texts)} документов в наш корпус.")
print(f"Тестовый вопрос: '{test_query}'")
print(f"Эталонный (True) ответ для оценки: '{true_answer_for_query}'")
print("Входные данные готовы.")
print("-" * 25)

Загружено 5 документов в наш корпус.
Тестовый вопрос: 'Compare the consistency and environmental impact of solar power versus hydropower.'
Эталонный (True) ответ для оценки: 'Solar power's consistency varies with weather and time of day, requiring storage like batteries. Hydropower is generally reliable, but large dams have significant environmental impacts on ecosystems and communities, unlike solar power's primary impact being land use for panels.'
Входные данные готовы.
-------------------------


### 5. Ключевой компонент: Функция нарезки текста на чанки

LLM и модели эмбеддингов имеют ограничения на объём текста, который они могут обрабатывать за раз. Кроме того, поиск работает лучше, когда он проводится по небольшим, сфокусированным кускам текста, а не по целым большим документам.

**Нарезка на чанки** — это процесс разделения больших документов на более мелкие, возможно, пересекающиеся сегменты.

- **`chunk_size`**: Определяет примерный размер (здесь — в словах) каждого чанка.
- **`chunk_overlap`**: Задает, сколько слов с конца одного чанка также будут включены в начало следующего чанка. Это помогает не потерять важную информацию, если она находится на границе между двумя чанками.

Мы определяем функцию `chunk_text` для такого разбиения на основе количества слов.

In [7]:
def chunk_text(text, chunk_size, chunk_overlap):
    """Разбивает один текстовый документ на пересекающиеся чанки по количеству слов.

    Args:
        text (str): Входной текст для нарезки.
        chunk_size (int): Желаемое количество слов в одном чанке.
        chunk_overlap (int): Количество слов, которое будет пересекаться между соседними чанками.

    Returns:
        list[str]: Список текстовых чанков.
    """
    words = text.split()      # Разделяем текст на список отдельных слов
    total_words = len(words)  # Считаем общее количество слов в тексте
    chunks = []               # Создаем пустой список для хранения чанков
    start_index = 0           # Начальный индекс слова для первого чанка

    # --- Проверка входных данных ---
    # Проверяем, что chunk_size — положительное целое число.
    if not isinstance(chunk_size, int) or chunk_size <= 0:
        print(f"  Внимание: Некорректный chunk_size ({chunk_size}). Должен быть положительным целым числом. Возвращаем весь текст одним чанком.")
        return [text]
    # Проверяем, что chunk_overlap — неотрицательное целое число, меньшее чем chunk_size.
    if not isinstance(chunk_overlap, int) or chunk_overlap < 0:
        print(f"  Внимание: Некорректный chunk_overlap ({chunk_overlap}). Должен быть неотрицательным целым числом. Перекрытие установлено в 0.")
        chunk_overlap = 0
    if chunk_overlap >= chunk_size:
        # Если перекрытие слишком большое, корректируем до разумной величины (например, 1/3 от chunk_size)
        # Это предотвращает бесконечные циклы или некорректную нарезку.
        adjusted_overlap = chunk_size // 3
        print(f"  Внимание: chunk_overlap ({chunk_overlap}) >= chunk_size ({chunk_size}). Перекрытие установлено в {adjusted_overlap}.")
        chunk_overlap = adjusted_overlap

    # --- Основной цикл нарезки ---
    # Продолжаем нарезку, пока start_index в пределах текста
    while start_index < total_words:
        # Определяем конечный индекс для текущего чанка.
        # Это минимум между (start + chunk_size) и общим количеством слов.
        end_index = min(start_index + chunk_size, total_words)

        # Извлекаем слова для текущего чанка и соединяем их обратно в строку.
        current_chunk_text = " ".join(words[start_index:end_index])
        chunks.append(current_chunk_text) # Добавляем сформированный чанк в список

        # Вычисляем начальный индекс для *следующего* чанка.
        # Продвигаемся на (chunk_size - chunk_overlap) слов вперёд.
        next_start_index = start_index + chunk_size - chunk_overlap

        # --- Проверки безопасности ---
        # Проверка 1: Предотвращение бесконечных циклов, если перекрытие мешает продвижению.
        # Такое возможно, если chunk_size очень маленький, а перекрытие слишком большое.
        if next_start_index <= start_index:
            if end_index == total_words: # Если уже в конце, можно выйти из цикла.
                break
            else:
                # Насильно двигаемся вперед хотя бы на одно слово.
                print(f"  Внимание: Логика нарезки застряла (start={start_index}, next_start={next_start_index}). Принудительно продолжаем.")
                next_start_index = start_index + 1

        # Проверка 2: Если рассчитанный следующий индекс уже на конце или за пределами текста — выходим.
        if next_start_index >= total_words:
            break

        # Сдвигаем start_index на следующую позицию для следующей итерации.
        start_index = next_start_index

    return chunks # Возвращаем итоговый список чанков

# --- Быстрый тест ---
# Тестируем функцию на первом документе с примерными параметрами.
print("Определена функция 'chunk_text'.")
sample_chunk_size = 150
sample_overlap = 30
sample_chunks = chunk_text(corpus_texts[0], sample_chunk_size, sample_overlap)
print(f"Тест нарезки на первом документе (размер={sample_chunk_size} слов, перекрытие={sample_overlap} слов): Создано чанков: {len(sample_chunks)}.")
if sample_chunks: # Печатаем только если чанки созданы
    print(f"Пример первого чанка:\n'{sample_chunks[0]}'")
print("-" * 25)

Определена функция 'chunk_text'.
Тест нарезки на первом документе (размер=150 слов, перекрытие=30 слов): Создано чанков: 1.
Пример первого чанка:
'Solar power uses PV panels or CSP systems. PV converts sunlight directly to electricity. CSP uses mirrors to heat fluid driving a turbine. It's clean but varies with weather/time. Storage (batteries) is key for consistency.'
-------------------------


### 6. Ключевой компонент: Подключение к Nebius AI

Для использования моделей Nebius AI (эмбеддинг, генерация, оценка) необходимо установить соединение с их API. Мы используем Python-библиотеку `openai`, которая предоставляет удобный способ взаимодействия с API, совместимыми с OpenAI, такими как Nebius.

Мы создаём объект клиента `OpenAI`, указывая наш API-ключ и конкретный URL-адрес эндпоинта Nebius API.

In [8]:
client = None # Инициализируем переменную клиента как None глобально

print("Пробуем инициализировать клиента Nebius AI...")
try:
    # Проверяем, действительно ли API-ключ доступен перед созданием клиента
    if not NEBIUS_API_KEY:
        raise ValueError("Отсутствует Nebius API Key. Невозможно инициализировать клиента.")

    # Создаем объект клиента OpenAI, настроенный для API Nebius.
    client = OpenAI(
        api_key=NEBIUS_API_KEY,     # Передаём ранее загруженный API-ключ
        base_url=NEBIUS_BASE_URL    # Указываем эндпоинт API Nebius
    )

    # Необязательно: Можно добавить быструю тестовую команду для проверки соединения с клиентом,
    # например, вывод списка моделей (если поддерживается и нужно). Может повлечь расходы.
    # try:
    #     client.models.list()
    #     print("Соединение с клиентом успешно проверено через список моделей.")
    # except Exception as test_e:
    #     print(f"Внимание: Не удалось проверить соединение с клиентом через тестовый вызов: {test_e}")

    print("Клиент Nebius AI успешно инициализирован. Готовы к работе с API.")

except Exception as e:
    # Обработка любых ошибок во время инициализации клиента (например, неверный ключ, проблемы с сетью)
    print(f"Ошибка при инициализации клиента Nebius AI: {e}")
    print("!!! Невозможно продолжить выполнение без корректного клиента. Пожалуйста, проверьте API-ключ и соединение с сетью. !!!")
    # Устанавливаем client обратно в None, чтобы избежать дальнейших попыток при неудачной инициализации
    client = None

print("Шаг по настройке клиента завершён.")
print("-" * 25)

Пробуем инициализировать клиента Nebius AI...
Клиент Nebius AI успешно инициализирован. Готовы к работе с API.
Шаг по настройке клиента завершён.
-------------------------


### 7. Ключевой компонент: Функция косинусного сходства

Чтобы оценить, насколько сгенерированный ответ семантически похож на наш эталонный ответ, мы используем **косинусное сходство**. Эта метрика измеряет косинус угла между двумя векторами (в нашем случае — эмбеддинг-векторами двух ответов).

- Оценка **1** означает, что векторы направлены в одну сторону (максимальное сходство).
- Оценка **0** означает, что векторы ортогональны (нет сходства).
- Оценка **-1** означает, что векторы направлены в противоположные стороны (максимальное различие).

Для текстовых эмбеддингов оценки обычно варьируются от 0 до 1, где большие значения означают большее семантическое сходство.

Мы определяем функцию `calculate_cosine_similarity`, которая принимает две текстовые строки, генерирует их эмбеддинги с помощью клиента Nebius и возвращает оценку их косинусного сходства.

In [9]:
def calculate_cosine_similarity(text1, text2, client, embedding_model):
    """Вычисляет косинусное сходство между эмбеддингами двух текстов.

    Args:
        text1 (str): Первая текстовая строка.
        text2 (str): Вторая текстовая строка.
        client (OpenAI): Инициализированный клиент Nebius AI.
        embedding_model (str): Название используемой модели эмбеддингов.

    Returns:
        float: Оценка косинусного сходства (от 0.0 до 1.0) или 0.0 в случае ошибки.
    """
    if not client:
        print("  Ошибка: Клиент Nebius недоступен для расчёта сходства.")
        return 0.0
    if not text1 or not text2:
        # Обработка случая, когда одна или обе строки пустые или None
        return 0.0

    try:
        # Генерируем эмбеддинги для обоих текстов одним запросом к API, если возможно
        response = client.embeddings.create(model=embedding_model, input=[text1, text2])

        # Извлекаем векторные представления
        embedding1 = np.array(response.data[0].embedding)
        embedding2 = np.array(response.data[1].embedding)

        # Преобразуем векторы к формату 2D, как ожидает функция cosine_similarity
        embedding1 = embedding1.reshape(1, -1)
        embedding2 = embedding2.reshape(1, -1)

        # Считаем косинусное сходство с помощью scikit-learn
        # cosine_similarity возвращает 2D-массив, например, [[значение]], поэтому извлекаем само значение.
        similarity_score = cosine_similarity(embedding1, embedding2)[0][0]

        # Ограничиваем значение в диапазоне от 0.0 до 1.0 для стабильности и единообразия
        return max(0.0, min(1.0, similarity_score))

    except Exception as e:
        print(f"  Ошибка при вычислении косинусного сходства: {e}")
        return 0.0 # Возвращаем 0.0 в случае любой ошибки API или вычислений

# --- Быстрый тест ---
print("Определена функция 'calculate_cosine_similarity'.")
if client: # Запуск теста только если клиент инициализирован
    test_sim = calculate_cosine_similarity("apple", "orange", client, NEBIUS_EMBEDDING_MODEL)
    print(f"Тест функции сходства: Сходство между 'apple' и 'orange' = {test_sim:.2f}")
else:
    print("Пропускаем тест функции сходства, так как клиент Nebius не инициализирован.")
print("-" * 25)

Определена функция 'calculate_cosine_similarity'.
Тест функции сходства: Сходство между 'apple' и 'orange' = 0.76
-------------------------


### 8. Эксперимент: Перебор конфигураций

В этом разделе находится основной экспериментальный цикл. Мы систематически переберём все комбинации настраиваемых параметров, определённых ранее (`CHUNK_SIZES_TO_TEST`, `CHUNK_OVERLAPS_TO_TEST`, `RETRIEVAL_TOP_K_TO_TEST`).

**Рабочий процесс для каждой комбинации параметров:**

1.  **Подготовка данных (Чанкирование/Эмбеддинг/Индексация — Шаг 8.1):**
    *   **Проверка необходимости перерасчёта:** Если параметры `chunk_size` или `chunk_overlap` изменились по сравнению с предыдущей итерацией, нужно повторно обработать корпус.
    *   **Нарезка на чанки:** Разделяем все документы из `corpus_texts` с текущими `chunk_size` и `chunk_overlap` с помощью функции `chunk_text`.
    *   **Эмбеддинг:** Преобразуем каждый чанк текста в числовой вектор (эмбеддинг) с помощью выбранной модели Nebius (`NEBIUS_EMBEDDING_MODEL`). Для эффективности делаем это пакетно.
    *   **Индексация:** Строим индекс FAISS (`IndexFlatL2`) из полученных эмбеддингов. FAISS позволяет очень быстро находить чанки, чьи эмбеддинги наиболее похожи на эмбеддинг запроса.
    *   *Оптимизация:* Если параметры чанкирования не изменились, используем уже созданные чанки, эмбеддинги и индекс из предыдущей итерации для экономии времени и API-запросов.

2.  **Тестирование стратегий RAG (Шаг 8.2):**
    *   Для текущего значения `top_k` выполняем каждую из определённых стратегий RAG:
        *   **Simple RAG:** Извлекаем `top_k` чанков на основе схожести с исходным запросом.
        *   **Query Rewrite RAG:** Сначала просим LLM переформулировать исходный запрос для более эффективного поиска. Затем извлекаем `top_k` чанков по схожести с *переформулированным* запросом.
        *   **Rerank RAG (симуляция):** Сначала извлекаем больше чанков (`top_k * RERANK_RETRIEVAL_MULTIPLIER`). Затем *симулируем* повторное ранжирование, просто выбирая топ-`top_k` результатов из этого большего множества. (В реальной реализации для этого используется отдельная модель для ранжирования).

3.  **Оценка и сохранение результатов (Шаг 8.3 внутри `run_and_evaluate`):**
    *   Для каждого запуска стратегии:
        *   **Извлечение:** Находим индексы релевантных чанков через индекс FAISS.
        *   **Генерация:** Формируем промпт с выбранными чанками в качестве контекста и *исходным* `test_query`. Отправляем это в модель генерации Nebius (`NEBIUS_GENERATION_MODEL`) для получения финального ответа.
        *   **Оценка (Faithfulness):** Используем оценщик LLM (`NEBIUS_EVALUATION_MODEL`) с `FAITHFULNESS_PROMPT`, чтобы оценить, насколько хорошо сгенерированный ответ соответствует `true_answer_for_query`.
        *   **Оценка (Relevancy):** Используем оценщик LLM с `RELEVANCY_PROMPT`, чтобы оценить, насколько хорошо сгенерированный ответ отвечает на `test_query`.
        *   **Оценка (Similarity):** Используем функцию `calculate_cosine_similarity` для оценки семантического сходства между сгенерированным ответом и `true_answer_for_query`.
        *   **Расчёт среднего балла:** Вычисляем среднее значение по оценкам Faithfulness, Relevancy и Similarity.
        *   **Сохранение:** Записываем все параметры (`chunk_size`, `overlap`, `top_k`, `strategy`), извлечённые индексы, переформулированный запрос (если есть), сгенерированный ответ, отдельные оценки, средний балл и время выполнения для этого запуска.

Для внешнего цикла по параметрам используем `tqdm` для отображения прогресс-бара.


In [10]:
# Список для хранения детальных результатов каждого эксперимента
all_results = []

# --- Переменные кэша для чанкирования/эмбеддинга/индексации ---
# Эти переменные помогают избежать лишних вычислений, если меняется только 'top_k'.
last_chunk_size = -1      # Хранит chunk_size, использованный в предыдущей итерации
last_overlap = -1         # Хранит chunk_overlap, использованный в предыдущей итерации
current_index = None      # Содержит текущий индекс FAISS
current_chunks = []       # Содержит список чанков для текущих настроек
current_embeddings = None # Содержит массив эмбеддингов для текущих чанков

# Проверяем, был ли клиент Nebius инициализирован перед началом работы
if not client:
    print("ОСТАНОВКА: Клиент Nebius AI не инициализирован. Невозможно запустить эксперимент.")
else:
    print("=== Запуск эксперимента RAG ===\n")

    # Создаем все возможные комбинации настраиваемых параметров
    param_combinations = list(itertools.product(
        CHUNK_SIZES_TO_TEST,
        CHUNK_OVERLAPS_TO_TEST,
        RETRIEVAL_TOP_K_TO_TEST
    ))

    print(f"Всего комбинаций параметров для тестирования: {len(param_combinations)}")

    # --- Основной цикл ---
    # Перебираем каждую комбинацию (chunk_size, chunk_overlap, top_k)
    # Используем tqdm для отображения прогресс-бара.
    for chunk_size, chunk_overlap, top_k in tqdm(param_combinations, desc="Тестирование конфигураций"):

        # --- 8.1 Обработка конфигурации чанкирования ---
        # Проверяем, изменились ли параметры чанкирования, чтобы решить, нужно ли пересчитывать.
        if chunk_size != last_chunk_size or chunk_overlap != last_overlap:
            # Раскомментируйте строку ниже для подробного логирования
            # print(f"\n--- Новая конфигурация чанкирования: Size={chunk_size}, Overlap={chunk_overlap} ---")

            # Обновляем кэш-переменные
            last_chunk_size, last_overlap = chunk_size, chunk_overlap
            # Сбрасываем индекс, чанки и эмбеддинги для новой конфигурации
            current_index = None
            current_chunks = []
            current_embeddings = None

            # --- 8.1a: Чанкирование ---
            # Применяем функцию chunk_text к каждому документу корпуса
            try:
                # print("  Нарезка документов на чанки...") # Раскомментируйте для подробного логирования
                temp_chunks = []
                for doc_index, doc in enumerate(corpus_texts):
                    doc_chunks = chunk_text(doc, chunk_size, chunk_overlap)
                    if not doc_chunks:
                         print(f"  Внимание: Не удалось создать чанки для документа {doc_index} с размером={chunk_size}, перекрытием={chunk_overlap}. Пропускаем документ.")
                         continue
                    temp_chunks.extend(doc_chunks)

                current_chunks = temp_chunks
                if not current_chunks:
                    # Если вообще не удалось создать ни одного чанка (например, из-за неверных параметров или пустого корпуса)
                    raise ValueError("Для данной конфигурации не создано ни одного чанка.")
                # print(f"    Создано всего чанков: {len(current_chunks)}.") # Раскомментируйте для подробного логирования
            except Exception as e:
                 print(f"    ОШИБКА при нарезке чанков для Size={chunk_size}, Overlap={chunk_overlap}: {e}. Пропускаем эту конфигурацию.")
                 last_chunk_size, last_overlap = -1, -1 # Сброс состояния кэша
                 continue # Переходим к следующей комбинации параметров

            # --- 8.1b: Эмбеддинг ---
            # Генерируем эмбеддинги для всех чанков через API Nebius.
            # print("  Генерация эмбеддингов...") # Раскомментируйте для подробного логирования
            try:
                batch_size = 32 # Обрабатываем чанки пакетами, чтобы не перегружать API.
                temp_embeddings = [] # Временный список для хранения эмбеддингов

                # Обрабатываем чанки пакетами
                for i in range(0, len(current_chunks), batch_size):
                    batch_texts = current_chunks[i : min(i + batch_size, len(current_chunks))]
                    # Запрос к API Nebius для текущего пакета
                    response = client.embeddings.create(model=NEBIUS_EMBEDDING_MODEL, input=batch_texts)
                    # Извлекаем эмбеддинги из ответа API
                    batch_embeddings = [item.embedding for item in response.data]
                    temp_embeddings.extend(batch_embeddings)
                    time.sleep(0.05) # Добавляем небольшую задержку между пакетами из вежливости к API.

                # Преобразуем список эмбеддингов в единый массив NumPy
                current_embeddings = np.array(temp_embeddings)
                # Базовая проверка массива эмбеддингов
                if current_embeddings.ndim != 2 or current_embeddings.shape[0] != len(current_chunks):
                    raise ValueError(f"Форма массива эмбеддингов не совпадает. Ожидалось ({len(current_chunks)}, dim), получено {current_embeddings.shape}")
                # print(f"    Сгенерировано эмбеддингов: {current_embeddings.shape[0]} (размерность: {current_embeddings.shape[1]}).") # Раскомментируйте для подробного логирования

            except Exception as e:
                print(f"    ОШИБКА при генерации эмбеддингов для Size={chunk_size}, Overlap={chunk_overlap}: {e}. Пропускаем эту конфигурацию чанков.")
                # Сбрасываем переменные кэша, чтобы показать неудачу для этой настройки
                last_chunk_size, last_overlap = -1, -1
                current_chunks = []
                current_embeddings = None
                continue # Пропускаем к следующей комбинации параметров

            # --- 8.1c: Индексация ---
            # Строим индекс FAISS для быстрого поиска по сходству.
            # print("  Строим поисковый индекс FAISS...") # Раскомментируйте для подробного логирования
            try:
                embedding_dim = current_embeddings.shape[1] # Получаем размерность эмбеддингов
                # Используем IndexFlatL2, который реализует точный поиск по L2 (евклидово расстояние).
                # Для современных эмбеддинговых моделей часто лучше работает косинусное сходство,
                # но IndexFlatIP (Inner Product) у FAISS с нормализованными векторами эквивалентен L2.
                current_index = faiss.IndexFlatL2(embedding_dim)
                # Добавляем эмбеддинги чанков в индекс. FAISS требует тип float32.
                current_index.add(current_embeddings.astype('float32'))

                if current_index.ntotal == 0:
                     raise ValueError("Индекс FAISS пуст после добавления векторов. Векторы не были добавлены.")
                # print(f"    Индекс FAISS готов с {current_index.ntotal} векторами.") # Раскомментируйте для подробного логирования
            except Exception as e:
                print(f"    ОШИБКА при построении индекса FAISS для Size={chunk_size}, Overlap={chunk_overlap}: {e}. Пропускаем эту конфигурацию чанков.")
                # Сброс переменных для индикации неудачи
                last_chunk_size, last_overlap = -1, -1
                current_index = None
                current_embeddings = None
                current_chunks = []
                continue # Переход к следующей комбинации параметров

        # --- 8.2 Тестирование стратегий RAG для текущего Top-K ---
        # Если дошли сюда — у нас валидные индекс и чанки для текущих chunk_size/overlap.

        # Проверяем, действительно ли индекс и чанки доступны (для безопасности)
        if current_index is None or not current_chunks:
            print(f"    ВНИМАНИЕ: Индекс или чанки недоступны для Size={chunk_size}, Overlap={chunk_overlap}. Пропускаем тест Top-K={top_k}.")
            continue

        # --- 8.3 Запуск и оценка одной стратегии RAG ---
        # Определяем вложенную функцию для основных шагов RAG (извлечение, генерация, оценка)
        # Это позволяет не дублировать код для каждой стратегии.
        def run_and_evaluate(strategy_name, query_to_use, k_retrieve, use_simulated_rerank=False):
            # print(f"    Запуск: {strategy_name} (k={k_retrieve}) ...") # Раскомментируйте для подробного логирования
            run_start_time = time.time() # Время старта для замера времени выполнения

            # Словарь для хранения результатов конкретного запуска
            result = {
                'chunk_size': chunk_size, 'overlap': chunk_overlap, 'top_k': k_retrieve,
                'strategy': strategy_name,
                'retrieved_indices': [], 'rewritten_query': None, 'answer': 'Error: Execution Failed',
                'faithfulness': 0.0, 'relevancy': 0.0, 'similarity_score': 0.0, 'avg_score': 0.0,
                'time_sec': 0.0
            }
            # Сохраняем переписанный запрос, если применимо
            if strategy_name == "Query Rewrite RAG":
                result['rewritten_query'] = query_to_use

            try:
                # --- Извлечение ---
                k_for_search = k_retrieve # Сколько чанков извлекать изначально
                if use_simulated_rerank:
                    # Для симулированного rerank извлекаем больше кандидатов сначала
                    k_for_search = k_retrieve * RERANK_RETRIEVAL_MULTIPLIER
                    # print(f"      Rerank: Первичное извлечение {k_for_search} кандидатов.") # Раскомментируйте для логирования

                # 1. Эмбеддинг запроса (оригинального или переписанного)
                query_embedding_response = client.embeddings.create(model=NEBIUS_EMBEDDING_MODEL, input=[query_to_use])
                query_embedding = query_embedding_response.data[0].embedding
                query_vector = np.array([query_embedding]).astype('float32') # FAISS требует float32

                # 2. Поиск в индексе FAISS
                # Гарантируем, что k не превышает количество элементов в индексе
                actual_k = min(k_for_search, current_index.ntotal)
                if actual_k == 0:
                    raise ValueError("Индекс пуст или k_for_search равно нулю, поиск невозможен.")

                # `current_index.search` возвращает расстояния и индексы ближайших соседей
                distances, indices = current_index.search(query_vector, actual_k)

                # 3. Обработка извлечённых индексов
                # Индексы могут содержать -1 если найдено меньше, чем 'actual_k' (для IndexFlatL2 не бывает, если k <= ntotal)
                retrieved_indices_all = indices[0]
                valid_indices = retrieved_indices_all[retrieved_indices_all != -1].tolist()

                # 4. Симулируем reranking (если применимо)
                # В этой симуляции просто берём топ k_retrieve результатов из большего множества.
                if use_simulated_rerank:
                    final_indices = valid_indices[:k_retrieve]
                    # print(f"      Rerank: Выбрано топ {len(final_indices)} индексов после симулированного rerank.") # Логирование
                else:
                    final_indices = valid_indices # Используем все валидные индексы до k_retrieve

                result['retrieved_indices'] = final_indices

                # 5. Получаем текстовые чанки по финальным индексам
                retrieved_chunks = [current_chunks[i] for i in final_indices]

                # Обрабатываем случай, когда ни одного чанка не найдено (редко, но возможно)
                if not retrieved_chunks:
                    print(f"      Внимание: Не найдено релевантных чанков для {strategy_name} (C={chunk_size}, O={chunk_overlap}, K={k_retrieve}). Ответ будет соответствующим.")
                    result['answer'] = "По запросу не найдено релевантного контекста в документах."
                    # Оценки оставляем по умолчанию (0.0), т.к. не было ответа из контекста
                else:
                    # --- Генерация ---
                    # Собираем чанки в единую строку-контекст
                    context_str = "\n\n".join(retrieved_chunks)

                    # Системный промпт для LLM генерации
                    sys_prompt_gen = "You are a helpful AI assistant. Answer the user's query based strictly on the provided context. If the context doesn't contain the answer, state that clearly. Be concise."

                    # Собираем пользовательский промпт с контекстом и оригинальным запросом
                    # Здесь важно использовать именно оригинальный запрос для генерации финального ответа, даже если для извлечения использовался переписанный.
                    user_prompt_gen = f"Context:\n------\n{context_str}\n------\n\nQuery: {test_query}\n\nAnswer:"

                    # Запрос к Nebius generation model
                    gen_response = client.chat.completions.create(
                        model=NEBIUS_GENERATION_MODEL,
                        messages=[
                            {"role": "system", "content": sys_prompt_gen},
                            {"role": "user", "content": user_prompt_gen}
                        ],
                        temperature=GENERATION_TEMPERATURE,
                        max_tokens=GENERATION_MAX_TOKENS,
                        top_p=GENERATION_TOP_P
                    )
                    # Извлекаем сгенерированный текст-ответ
                    generated_answer = gen_response.choices[0].message.content.strip()
                    result['answer'] = generated_answer
                    # print(f"      Сгенерированный ответ: {generated_answer[:100].replace('\n', ' ')}...") # Опционально

                    # --- Оценка ---
                    # Оцениваем сгенерированный ответ по Faithfulness, Relevancy, Similarity
                    # print(f"      Оценка ответа... (Faithfulness, Relevancy, Similarity)") # Опционально

                    # Параметры для оценки (температура — 0.0 для детерминированности)
                    eval_params = {'model': NEBIUS_EVALUATION_MODEL, 'temperature': 0.0, 'max_tokens': 10}

                    # 1. Оценка Faithfulness
                    prompt_f = FAITHFULNESS_PROMPT.format(question=test_query, response=generated_answer, true_answer=true_answer_for_query)
                    try:
                        resp_f = client.chat.completions.create(messages=[{"role": "user", "content": prompt_f}], **eval_params)
                        # Парсим оценку, приводим к диапазону 0.0–1.0
                        result['faithfulness'] = max(0.0, min(1.0, float(resp_f.choices[0].message.content.strip())))
                    except Exception as eval_e:
                        print(f"      Внимание: Ошибка разбора Faithfulness score для {strategy_name} - {eval_e}. Оценка выставлена в 0.0")
                        result['faithfulness'] = 0.0

                    # 2. Оценка Relevancy
                    prompt_r = RELEVANCY_PROMPT.format(question=test_query, response=generated_answer)
                    try:
                        resp_r = client.chat.completions.create(messages=[{"role": "user", "content": prompt_r}], **eval_params)
                        # Парсим оценку, приводим к диапазону 0.0–1.0
                        result['relevancy'] = max(0.0, min(1.0, float(resp_r.choices[0].message.content.strip())))
                    except Exception as eval_e:
                        print(f"      Внимание: Ошибка разбора Relevancy score для {strategy_name} - {eval_e}. Оценка выставлена в 0.0")
                        result['relevancy'] = 0.0

                    # 3. Расчет Similarity
                    result['similarity_score'] = calculate_cosine_similarity(
                        generated_answer,
                        true_answer_for_query,
                        client,
                        NEBIUS_EMBEDDING_MODEL
                    )

                    # 4. Вычисляем средний балл (Faithfulness, Relevancy, Similarity)
                    result['avg_score'] = (result['faithfulness'] + result['relevancy'] + result['similarity_score']) / 3.0

            except Exception as e:
                # Обработка любых неожиданных ошибок в процессе извлечения/генерации/оценки
                error_message = f"ОШИБКА при {strategy_name} (C={chunk_size}, O={chunk_overlap}, K={k_retrieve}): {str(e)[:200]}..."
                print(f"    {error_message}")
                result['answer'] = error_message # Сохраняем ошибку в поле ответа
                # Оставляем оценки в состоянии по умолчанию (0.0)
                result['faithfulness'] = 0.0
                result['relevancy'] = 0.0
                result['similarity_score'] = 0.0
                result['avg_score'] = 0.0

            # Записываем общее время выполнения для данного запуска
            run_end_time = time.time()
            result['time_sec'] = run_end_time - run_start_time

            # Печатаем краткую информацию о запуске (для отслеживания прогресса)
            print(f"    Готово: {strategy_name} (C={chunk_size}, O={chunk_overlap}, K={k_retrieve}). Средний балл={result['avg_score']:.2f}, Время={result['time_sec']:.2f}с")
            return result
        # --- Конец вложенной функции run_and_evaluate ---

        # --- Выполняем стратегии RAG с помощью run_and_evaluate ---

        # Стратегия 1: Simple RAG (используем оригинальный запрос для поиска)
        result_simple = run_and_evaluate("Simple RAG", test_query, top_k)
        all_results.append(result_simple)

        # Стратегия 2: Query Rewrite RAG
        rewritten_q = test_query # По умолчанию используем оригинальный запрос, если rewrite не удался
        try:
             # print("    Переписываем запрос для Rewrite RAG...") # Опционально
             # Промпты для задачи переписывания запроса
             sys_prompt_rw = "You are an expert query optimizer. Rewrite the user's query to be ideal for vector database retrieval. Focus on key entities, concepts, and relationships. Remove conversational fluff. Output ONLY the rewritten query text."
             user_prompt_rw = f"Original Query: {test_query}\n\nRewritten Query:"

             # Запрос к LLM на переписывание запроса
             resp_rw = client.chat.completions.create(
                 model=NEBIUS_GENERATION_MODEL, # Для этой задачи тоже подойдёт модель генерации
                 messages=[
                     {"role": "system", "content": sys_prompt_rw},
                     {"role": "user", "content": user_prompt_rw}
                 ],
                 temperature=0.1, # Низкая температура для лаконичного результата
                 max_tokens=100,
                 top_p=0.9
             )
             # Очищаем ответ LLM, чтобы получить только текст запроса
             candidate_q = resp_rw.choices[0].message.content.strip()
             # Убираем возможные префиксы типа "Rewritten Query:" или "Query:"
             candidate_q = re.sub(r'^(rewritten query:|query:)\s*', '', candidate_q, flags=re.IGNORECASE).strip('"')

             # Используем переписанный запрос, только если он реально отличается и не слишком короткий
             if candidate_q and len(candidate_q) > 5 and candidate_q.lower() != test_query.lower():
                 rewritten_q = candidate_q
                 # print(f"      Используем переписанный запрос: '{rewritten_q}'") # Опционально
             # else:
                 # print("      Переписывание не удалось или результат такой же, как оригинал. Используем оригинальный запрос.") # Опционально
        except Exception as e:
             print(f"    Внимание: Ошибка при переписывании запроса: {e}. Используем оригинальный запрос.")
             rewritten_q = test_query # В случае ошибки возвращаемся к оригинальному запросу

        # Оцениваем результат с (возможно) переписанным запросом для поиска
        result_rewrite = run_and_evaluate("Query Rewrite RAG", rewritten_q, top_k)
        all_results.append(result_rewrite)

        # Стратегия 3: Rerank RAG (симуляция)
        # Используем оригинальный запрос для поиска, но симулируем процесс ранжирования
        result_rerank = run_and_evaluate("Rerank RAG (Simulated)", test_query, top_k, use_simulated_rerank=True)
        all_results.append(result_rerank)

    print("\n=== Экспериментальный цикл RAG завершён ===")
    print("-" * 25)

=== Запуск эксперимента RAG ===

Всего комбинаций параметров для тестирования: 8


Тестирование конфигураций:   0%|          | 0/8 [00:00<?, ?it/s]

    Готово: Simple RAG (C=150, O=30, K=3). Средний балл=0.89, Время=7.41с
      Внимание: Ошибка разбора Relevancy score для Query Rewrite RAG - could not convert string to float: '0.9\n\nThe AI response is highly relevant'. Оценка выставлена в 0.0
    Готово: Query Rewrite RAG (C=150, O=30, K=3). Средний балл=0.59, Время=4.94с
    Готово: Rerank RAG (Simulated) (C=150, O=30, K=3). Средний балл=0.90, Время=7.27с
    Готово: Simple RAG (C=150, O=30, K=5). Средний балл=0.89, Время=6.04с
    Готово: Query Rewrite RAG (C=150, O=30, K=5). Средний балл=0.89, Время=14.73с
    Готово: Rerank RAG (Simulated) (C=150, O=30, K=5). Средний балл=0.89, Время=10.75с
    Готово: Simple RAG (C=150, O=50, K=3). Средний балл=0.89, Время=6.65с
    Готово: Query Rewrite RAG (C=150, O=50, K=3). Средний балл=0.89, Время=6.43с
    Готово: Rerank RAG (Simulated) (C=150, O=50, K=3). Средний балл=0.90, Время=6.64с
    Готово: Simple RAG (C=150, O=50, K=5). Средний балл=0.89, Время=8.76с
    Готово: Query Rewrite 

### 9. Анализ: просмотр результатов

Теперь, когда экспериментальный цикл завершён и в `all_results` содержатся данные каждого прогона, мы используем библиотеку Pandas для анализа результатов.

1.  **Создание DataFrame:** Преобразуем список словарей результатов (`all_results`) в DataFrame Pandas для удобной обработки и просмотра.
2.  **Сортировка результатов:** Отсортируем DataFrame по `avg_score` (среднему значению Faithfulness, Relevancy и Similarity) по убыванию, чтобы лучшие конфигурации были вверху.
3.  **Вывод топ-конфигураций:** Отобразим верхние N строк отсортированного DataFrame с ключевыми параметрами, оценками и сгенерированным ответом — это поможет быстро выделить наиболее перспективные настройки.
4.  **Сводка лучшего запуска:** Напечатаем чёткое резюме единственной самой успешной конфигурации по среднему баллу: её параметры, индивидуальные оценки, затраченное время и полный сгенерированный ответ.

In [11]:
print("--- Анализ результатов эксперимента ---")

# Сначала проверяем, были ли вообще собраны какие-либо результаты
if not all_results:
    print("В ходе эксперимента не было получено результатов. Анализ невозможен.")
else:
    # Преобразуем список словарей с результатами в DataFrame Pandas
    results_df = pd.DataFrame(all_results)
    print(f"Всего собрано результатов: {len(results_df)}")

    # Сортируем DataFrame по столбцу 'avg_score' по убыванию (лучшие — сверху)
    # Используем reset_index(drop=True) для чистого индекса с нуля после сортировки.
    results_df_sorted = results_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)

    print("\n--- Топ-10 конфигураций (отсортировано по среднему баллу) ---")
    # Определяем столбцы, которые хотим показать в итоговой таблице
    display_cols = [
        'chunk_size', 'overlap', 'top_k', 'strategy',
        'avg_score', 'faithfulness', 'relevancy', 'similarity_score', # Добавлено similarity
        'time_sec',
        'answer' # Включаем ответ для качественной оценки лучших запусков
    ]
    # Исключаем те столбцы, которых нет (например, если была ошибка и не все были заполнены)
    display_cols = [col for col in display_cols if col in results_df_sorted.columns]

    # Показываем первые 10 строк отсортированного DataFrame по выбранным столбцам
    # Функция display() даёт красивый вывод в Jupyter.
    display(results_df_sorted[display_cols].head(10))

    # --- Сводка по лучшей конфигурации ---
    print("\n--- Сводка по лучшей конфигурации ---")
    # Проверяем, что DataFrame после сортировки не пустой
    if not results_df_sorted.empty:
        # Берём первую строку (индекс 0) — это конфигурация с лучшим баллом
        best_run = results_df_sorted.iloc[0]

        # Выводим параметры и результаты лучшей конфигурации
        print(f"Размер чанка: {best_run.get('chunk_size', 'N/A')} слов")
        print(f"Перекрытие: {best_run.get('overlap', 'N/A')} слов")
        print(f"Top-K извлечённых: {best_run.get('top_k', 'N/A')} чанков")
        print(f"Стратегия: {best_run.get('strategy', 'N/A')}")
        # Для устойчивости используем .get(col, default) — если столбца нет
        avg_score = best_run.get('avg_score', 0.0)
        faithfulness = best_run.get('faithfulness', 0.0)
        relevancy = best_run.get('relevancy', 0.0)
        similarity = best_run.get('similarity_score', 0.0)
        time_sec = best_run.get('time_sec', 0.0)
        best_answer = best_run.get('answer', 'N/A')

        print(f"---> Средний балл (Faith+Rel+Sim): {avg_score:.3f}")
        print(f"      (Достоверность: {faithfulness:.3f}, Релевантность: {relevancy:.3f}, Сходство: {similarity:.3f})")
        print(f"Затраченное время: {time_sec:.2f} секунд")
        print(f"\nЛучший сгенерированный ответ:")
        # Печатаем полный ответ, сгенерированный лучшей конфигурацией
        print(best_answer)
    else:
        # Обработка случая, когда не было ни одного валидного результата
        print("Не удалось определить лучшую конфигурацию (не найдено валидных результатов).")

print("\n--- Анализ завершён --- ")

--- Анализ результатов эксперимента ---
Всего собрано результатов: 24

--- Топ-10 конфигураций (отсортировано по среднему баллу) ---


,chunk_size,overlap,top_k,strategy,avg_score,faithfulness,relevancy,similarity_score,time_sec,answer
0,150,30,3,Rerank RAG (Simulated),0.897648,0.9,1.0,0.792944,7.266874,Solar power and hydropower differ significantly in consistency and environmental impact:\n\n**Consistency:**\n- **Hydropower** is highly reliable ...
1,250,50,5,Rerank RAG (Simulated),0.897204,0.9,1.0,0.791613,6.744190,Solar power and hydropower differ significantly in consistency and environmental impact:\n\n- **Consistency**: \n - **Solar Power**: Inconsisten...
2,150,50,3,Rerank RAG (Simulated),0.895689,0.9,1.0,0.787067,6.643735,"**Consistency:**\n- **Hydropower** is highly reliable and consistent, providing large-scale power 24/7, as it relies on the continuous flow of wat..."
3,250,50,3,Rerank RAG (Simulated),0.895561,0.9,1.0,0.786683,5.991125,"**Consistency:**\n- **Hydropower** is highly reliable and consistent, providing large-scale power 24/7, as it relies on the continuous flow of wat..."
4,250,30,3,Simple RAG,0.895335,0.9,1.0,0.786004,6.273028,"**Consistency:**\n- **Hydropower** is highly consistent and reliable, providing large-scale power 24/7, as it relies on the continuous flow of wat..."
5,150,50,3,Query Rewrite RAG,0.894385,0.9,1.0,0.783156,6.430842,"**Consistency:**\n- **Hydropower** is highly reliable and provides consistent, large-scale power, as it is not dependent on weather conditions and..."
6,250,30,3,Query Rewrite RAG,0.893745,0.9,1.0,0.781236,9.103239,"**Consistency:**\n- **Hydropower** is highly consistent and reliable, providing large-scale power 24/7, as it relies on the continuous flow of wat..."
7,250,50,3,Simple RAG,0.893461,0.9,1.0,0.780383,9.975372,"**Consistency:**\n- **Hydropower:** Highly reliable and consistent, as it can generate electricity continuously as long as water flow is maintaine..."
8,150,50,3,Simple RAG,0.892797,0.9,1.0,0.778390,6.650850,"**Consistency:**\n- **Hydropower** is highly reliable and provides consistent, large-scale power, as it is not dependent on weather conditions and..."
9,150,30,5,Rerank RAG (Simulated),0.892647,0.9,1.0,0.777942,10.745529,"**Consistency:** \n- **Solar Power:** Inconsistent due to dependence on weather and daylight. Requires storage solutions (e.g., batteries) for re..."



--- Сводка по лучшей конфигурации ---
Размер чанка: 150 слов
Перекрытие: 30 слов
Top-K извлечённых: 3 чанков
Стратегия: Rerank RAG (Simulated)
---> Средний балл (Faith+Rel+Sim): 0.898
      (Достоверность: 0.900, Релевантность: 1.000, Сходство: 0.793)
Затраченное время: 7.27 секунд

Лучший сгенерированный ответ:
Solar power and hydropower differ significantly in consistency and environmental impact:

**Consistency:**
- **Hydropower** is highly reliable and consistent, providing large-scale power 24/7, as it relies on the continuous flow of water.
- **Solar power** is less consistent, as it depends on weather conditions and time of day. It requires storage solutions (like batteries) to ensure a steady supply.

**Environmental Impact:**
- **Hydropower** has significant environmental impacts, particularly from large dams, which can harm ecosystems, disrupt fish migration, and displace communities. Run-of-river systems are less disruptive but still affect local environments.
- **Solar pow

### 10. Выводы: Что мы узнали?

Мы успешно построили и выполнили сквозной пайплайн для экспериментов с различными конфигурациями RAG и оценки их эффективности по нескольким метрикам на платформе Nebius AI.

Анализируя таблицу результатов и сводку лучшей конфигурации выше, мы можем сделать выводы, актуальные именно для *нашего выбранного корпуса, запроса и моделей*.

**Вопросы для размышления:**

*   **Влияние нарезки на чанки:** Какой именно `chunk_size` или `overlap` чаще давал более высокие средние оценки? Почему небольшие чанки могут лучше захватывать отдельные факты, а большие — давать больше контекста? Как перекрытие повлияло на результаты?
*   **Количество извлекаемых чанков (`top_k`):** Как увеличение `top_k` отразилось на оценках? Приводило ли извлечение большего числа чанков всегда к лучшим ответам, или иногда добавляло "шум" и нерелевантную информацию, снижая достоверность или сходство?
*   **Сравнение стратегий:** Давали ли стратегии 'Query Rewrite' или 'Rerank (Simulated)' стабильное преимущество перед 'Simple RAG' по среднему баллу? Было ли улучшение достаточно значимым, чтобы оправдать дополнительные шаги (например, лишний вызов LLM для переписывания запроса, большее первоначальное извлечение для rerank)?
*   **Метрики оценки:**
    *   Посмотрите на 'Лучший ответ' и сравните его с `true_answer_for_query`. Насколько отдельные оценки (Faithfulness, Relevancy, Similarity) отражают ваше субъективное восприятие качества?
    *   Всегда ли высокая схожесть коррелировала с высокой достоверностью? Может ли ответ быть похожим, но недостоверным, или достоверным, но непохожим?
    *   Насколько надёжной вам кажется автоматическая оценка LLM (Faithfulness, Relevancy) по сравнению с более объективной косинусной мерой? Каковы потенциальные ограничения LLM-оценки (например, чувствительность к формулировке промпта, "предвзятость" модели)?
*   **Общая производительность:** Получилась ли у какой-либо конфигурации почти идеальная средняя оценка? Что может мешать получить идеальный результат (например, ограничения исходных документов, неоднозначность языка, неидеальный поиск)?

**Главный вывод:** Оптимизация системы RAG — это итеративный процесс. Лучшая конфигурация часто сильно зависит от конкретного датасета, типа пользовательских запросов, выбранных моделей эмбеддинга и LLM, а также критериев оценки. Системный эксперимент, как показано в этом ноутбуке, крайне важен для поиска оптимальных настроек под свою задачу.

**Возможные следующие шаги и направления развития:**

*   **Расширить диапазон параметров:** Попробовать больше значений для `chunk_size`, `overlap` и `top_k`.
*   **Разные типы запросов:** Протестировать те же конфигурации на других типах вопросов (например, факт, сравнение, саммари), чтобы посмотреть, как меняется эффективность.
*   **Больше/иной корпус:** Использовать более объёмную или тематическую базу знаний.
*   **Реализовать настоящий rerank:** Заменить симулированное ранжирование на полноценную модель-кросс-энкодер (например, из Hugging Face Transformers или Cohere Rerank) для повторной оценки релевантности документов.
*   **Альтернативные модели:** Поэкспериментировать с разными моделями Nebius AI для эмбеддинга, генерации или оценки.
*   **Продвинутое чанкирование:** Опробовать более сложные стратегии нарезки (например, рекурсивное разбиение по символам, семантическое чанкирование).
*   **Человеческая оценка:** Добавить экспертную оценку к автоматическим метрикам для более глубокого анализа качества ответов.